In [ ]:
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom, rotate
from scipy import signal

# 6. Image Restoration : deconvolution

In [ ]:
s = 8
board = np.kron([ [1, 0]* 4, [0, 1]*4 ]*4, np.ones((s,s)))
checkerboard = zoom(board, (8, 8), order=0)
checkerboard = checkerboard.astype(np.float32) / 255.0

plt.imshow(checkerboard, 'gray')
plt.axis('off')
plt.title('Checkerboard')

In [ ]:
# Define the size and direction of the motion blur
size = 10
direction = np.array([1, 1])

# Create a 2D array with a single row of ones in the center
kernel = np.zeros((size, size))
center = int((size - 1) / 2)
kernel[center, :] = np.ones(size)

# Normalize the kernel so that its sum is 1
kernel /= np.sum(kernel)

# Rotate the kernel to create the motion blur in the specified direction
kernel = rotate(kernel, np.rad2deg(np.arctan2(direction[1], direction[0])) - 90, reshape=False)

# Generate a PSF by convolving the kernel with itself
psf = signal.convolve2d(kernel, kernel)

# Normalize the PSF so that its sum is 1
psf /= np.sum(psf)

In [ ]:
def addGaussianNoise(I , sigma=1000):
    """
    Adds Gaussian noise to an image
    I: original image
    sigma: signal / noise ratio
    """
    I2 = I.copy()
    m=np.min(I)
    M=np.max(I)
    N = (M-m)/sigma*np.random.randn(I.shape[0], I .shape[1]) ;
    I2 = I2 + N
    I2 [I2>M] = M
    I2 [I2<m] = m
    return I2 , I - I2

In [ ]:
Cb = signal.convolve2d(checkerboard, psf2otf(psf,(8,8)) , boundary='wrap', mode='same');
plt.imshow(Cb, cmap='gray')
plt.title ('Motion blur + Gaussian noise on checkerboard')
plt.show()

In [ ]:
Cb = signal.convolve2d(checkerboard, psf , boundary='wrap', mode='same');
Cbn, noise = addGaussianNoise(Cb)
plt.imshow(Cb, cmap='gray')
plt.title ('Motion blur + Gaussian noise on checkerboard')
plt.show()

In [ ]:
def psf2otf ( psf , s ) :
    """
    Get OTF (Optical Transfer Function) from PSF ( Point Spread Function)
    OTF is basically the Fourier Transform of the PSF, centered
    psf : PSF
    s : shape of the result , zero-padding is used to center is Fourier Transform
    """
    sh = psf .shape;
    sh = np.array (sh) ;
    s = np.array ( s ) ;
    pad = s - sh;
    h = np.pad(psf , ((0, pad[0]) , (0, pad[1]) ) , mode='constant') ;
    shift = ( int (pad[0]/2+1) , int (pad[1]/2+1) ) ;
    h_centered = np. roll (h, tuple ( shift ) , axis =(0,1) ) ;
    h_centered = np. fft . fftshift (h_centered) ;
    H = np. fft . fft2 (h_centered, s ) ;
    H = np. real (H);
    return H;

In [ ]:
H = psf2otf(psf , checkerboard.shape) ;
G = np. fft . fft2 (Cb);
alpha = 0.001;
F = G / (H+alpha);
fr = np. real (np. fft . ifft2 (F) ) ;
plt .imshow(fr,'gray');
plt . title ( 'Noiseless (only motion blur ) direct reconstruction ' )
plt .show()